This notebook is for testing of:
- Data transformation from a "cleaned, preprocessed, splited" pickle file 
- Model loading
- Model training

In [9]:
PROJECT_DIR = "C:/Users/Uyen/Documents/nlp/Thesis/aspect-targeted-polarity-classification"

In [2]:
import sys
import os

sys.path.append(PROJECT_DIR)
sys.path.append(os.path.join(PROJECT_DIR, "finetuning_and_classification"))


import utils
from utils_glue import (compute_metrics, convert_examples_to_features,
                        output_modes, processors)


In [5]:
# Test data load from pkl
# Test function YNmilk_data_reform

sentences, aspect_term_sentiments =utils.YNmilk_data_reform("C:\\Users\\Uyen\\Documents\\nlp\\YNdata\\data_splited\\train.pickle")

sentences[0:5], aspect_term_sentiments[0:5]

(["' ngày uống được 3 cữ thì tốt nhất chị ạ '",
  "' Thơm ngon bổ_dưỡng '",
  "' Thơm ngon bổ_dưỡng '",
  "' Minh_Anh Leđừng quên ghé ngay cửa hàng Giấc mơ sữa Việt để mua ngay sữa Yoko để rinh những món quà đáng yêu cho bé nha😄 '",
  "' Hay quá '"],
 [[('Thành_phần', 'POS')],
  [('Dinh_dưỡng', 'POS')],
  [('Hương_vị', 'POS')],
  [('Quà tặng', 'POS')],
  [('Thương_mại_điện_tử', 'POS')]])

In [4]:
os.chdir(PROJECT_DIR)
!python prepare_YNmilk_datasets.py \
    --files "C:\\Users\\Uyen\\Documents\\nlp\YNdata\\data_splited\\train.pickle" \
    --output_dir data/transformed/yndata \
    --istrain

data/transformed/yndata
C:\\Users\\Uyen\\Documents\\nlp\YNdata\\data_splited\\train.pickle
Dataset: Train
#Sentences with minimum 1 label 178065
Label Counts [('POS', 145849), ('NEU', 29108), ('NEG', 3108)]
#SentencePairs 178065
POS% 0.8190773032319658
NEG% 0.01745430039592284
NEU% 0.1634683963721113
POS/NEG 46.926962676962674
POS/NEU 5.010615638312491
NEG/NEU 0.10677476982272914

Dataset: Dev
#Sentences with minimum 1 label 17806
Label Counts [('POS', 14652), ('NEU', 2850), ('NEG', 304)]
#SentencePairs 17806
POS% 0.822868695945187
NEG% 0.017072896776367516
NEU% 0.16005840727844547
POS/NEG 48.19736842105263
POS/NEU 5.1410526315789475
NEG/NEU 0.10666666666666667

Dataset: TrainSplit
#Sentences with minimum 1 label 160259
Label Counts [('POS', 131197), ('NEU', 26258), ('NEG', 2804)]
#SentencePairs 160259
POS% 0.8186560505182237
NEG% 0.01749667725369558
NEU% 0.1638472722280808
POS/NEG 46.78922967189729
POS/NEU 4.9964582222560745
NEG/NEU 0.10678650316094143



In [3]:
!python prepare_YNmilk_datasets.py \
--files "C:\\Users\\Uyen\\Documents\\nlp\YNdata\\data_splited\\validate.pickle" \
--output_dir data/transformed/yndata 

data/transformed/yndata
C:\\Users\\Uyen\\Documents\\nlp\YNdata\\data_splited\\validate.pickle
Dataset: Test
#Sentences with minimum 1 label 38427
Label Counts [('POS', 31555), ('NEU', 6166), ('NEG', 706)]
#SentencePairs 38427
POS% 0.8211674083326828
NEG% 0.018372498503656284
NEU% 0.16046009316366097
POS/NEG 44.695467422096314
POS/NEU 5.117580278949076
NEG/NEU 0.11449886474213429



In [21]:
os.chdir(PROJECT_DIR)

MODEL_DIR = os.path.join(PROJECT_DIR, "data/models/phobert_base/huggingface_phobert_base")
DATA_DIR = os.path.join(PROJECT_DIR, "data/transformed/yndata")
OUTPUT_DIR = os.path.join(PROJECT_DIR, "YNData-ar-phobert-milk")

!python ./finetuning_and_classification/run_glue.py \
    --model_type="phobert" \
    --model_name_or_path=C:/Users/Uyen/Documents/nlp/thesis/aspect-targeted-polarity-classification/data/models/phobert_base/huggingface_phobert_base \
    --do_train \
    # --evaluate_during_training \
    --logging_steps 100 --save_steps 200 --task_name="atsc" \
    --seed 42 \
    --do_lower_case \
    --data_dir="$DATA_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --max_seq_length=128 \
    --learning_rate 3e-5 \
    --per_gpu_eval_batch_size=32 \
    --gradient_accumulation_steps=1 \
    --max_steps=800 \
    --overwrite_output_dir --overwrite_cache --warmup_steps=120
    

03/02/2021 12:45:55 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at C:/Users/Uyen/Documents/nlp/thesis/aspect-targeted-polarity-classification/data/models/phobert_base/huggingface_phobert_base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model 

In [3]:
from transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer)


# config_class, model_class, tokenizer_class = (BertConfig, BertForSequenceClassification, BertTokenizer)
# config = config_class.from_pretrained(MODEL_DIR, num_labels=num_labels, finetuning_task=task_name)
# tokenizer = tokenizer_class.from_pretrained(MODEL_DIR, do_lower_case=True)
# model = model_class.from_pretrained(MODEL_DIR, from_tf=bool('.ckpt' in MODEL_DIR), config=config)


In [13]:
!python prepare_YNmilk_datasets.py \
--files "C:\\Users\\Uyen\\Documents\\nlp\YNdata\\data_splited\\test.pickle" \
--output_dir data/transformed/yndata/testdata

data/transformed/yndata/testdata
C:\\Users\\Uyen\\Documents\\nlp\YNdata\\data_splited\\test.pickle
Dataset: Test
#Sentences with minimum 1 label 38043
Label Counts [('POS', 30960), ('NEU', 6437), ('NEG', 646)]
#SentencePairs 38043
POS% 0.8138159451147385
NEG% 0.0169807849012959
NEU% 0.1692032699839655
POS/NEG 47.925696594427244
POS/NEU 4.80969395681218
NEG/NEU 0.10035730930557714



In [25]:
os.chdir(PROJECT_DIR)

!python ./finetuning_and_classification/run_glue.py \
    --model_type="phobert" \
    --do_eval \
    --model_name_or_path="C:/Users/Uyen/Documents/nlp/Thesis/aspect-targeted-polarity-classification/data/models/finetuned_phobert4classification/" \
    --task_name="atsc" \
    --output_dir="C:/Users/Uyen/Documents/nlp/Thesis/aspect-targeted-polarity-classification/data/models/finetuned_phobert4classification/"  \
    --do_lower_case \
    --data_dir=./data/transformed/yndata/testdata \
    --max_seq_length=128 \
    --per_gpu_eval_batch_size=32 \
    --overwrite_output_dir --overwrite_cache

Evaluating:  33%|███▎      | 391/1189 [03:45<07:41,  1.73it/s]

In [21]:
# text parser
import xml.etree.ElementTree as ET

filepath = "C:/Users/Uyen/Documents/nlp/thesis/aspect-targeted-polarity-classification/data/transformed/yndata/testdata/test.xml"

with open(filepath, 'rb') as f:
    sentence_elements = ET.parse(f).getroot().iter('sentence')

e_0 = list(sentence_elements)[0]
e_0.find('text').text, e_0.find('aspectTerms').find('aspectTerm').get('term')

("' Công_nhận uống Friso tốt cho hệ tiêu_hoá lắm luôn đó nha c \\ n '",
 'Chức_năng tiêu_hoá')

In [20]:
# Test dataloader
filepath = "C:/Users/Uyen/Documents/nlp/thesis/aspect-targeted-polarity-classification/data/transformed/yndata/testdata/test.xml"


'Chức_năng tiêu_hoá'

In [29]:
# !pip install datasets

from datasets import load_dataset

data_dir="C:/Users/Uyen/Documents/nlp/thesis/aspect-targeted-polarity-classification/data/transformed/yndata/"

dataset = load_dataset('load_dataset_ar.py', data_dir=data_dir)

Using custom data configuration default-aa233d0ff84f88a1
Dataset yn_dataset downloaded and prepared to C:\Users\Uyen\.cache\huggingface\datasets\yn_dataset\default-aa233d0ff84f88a1\1.1.0\976a607c4f36d10e025b4a64da7d52f8fe14b425fdc639546361d76033602814. Subsequent calls will reuse this data.


In [37]:
dataset['train']["labels"]

[32,
 13,
 6,
 31,
 27,
 33,
 22,
 3,
 3,
 35,
 24,
 32,
 22,
 17,
 32,
 3,
 31,
 3,
 29,
 30,
 19,
 5,
 3,
 30,
 31,
 24,
 6,
 13,
 17,
 6,
 28,
 32,
 17,
 3,
 28,
 24,
 6,
 3,
 17,
 32,
 12,
 29,
 13,
 13,
 24,
 5,
 5,
 5,
 16,
 13,
 24,
 6,
 3,
 3,
 35,
 6,
 3,
 4,
 32,
 6,
 30,
 3,
 3,
 25,
 5,
 24,
 24,
 24,
 3,
 6,
 35,
 24,
 6,
 17,
 24,
 6,
 24,
 6,
 3,
 24,
 6,
 22,
 30,
 3,
 25,
 24,
 27,
 33,
 24,
 3,
 3,
 0,
 5,
 13,
 3,
 24,
 6,
 24,
 31,
 13,
 24,
 32,
 12,
 19,
 24,
 25,
 36,
 5,
 6,
 16,
 24,
 19,
 12,
 3,
 28,
 29,
 31,
 3,
 17,
 28,
 13,
 32,
 19,
 24,
 32,
 12,
 24,
 3,
 24,
 6,
 25,
 34,
 19,
 17,
 3,
 18,
 33,
 6,
 3,
 6,
 25,
 3,
 6,
 29,
 6,
 3,
 26,
 25,
 3,
 24,
 30,
 32,
 3,
 19,
 32,
 19,
 30,
 24,
 32,
 12,
 32,
 10,
 3,
 3,
 3,
 24,
 32,
 17,
 28,
 19,
 32,
 5,
 19,
 6,
 12,
 19,
 20,
 14,
 3,
 6,
 25,
 24,
 35,
 25,
 16,
 27,
 33,
 6,
 23,
 3,
 6,
 17,
 6,
 25,
 6,
 3,
 16,
 6,
 3,
 16,
 5,
 27,
 33,
 27,
 33,
 13,
 29,
 3,
 25,
 3,
 36,
 29,
 24,
 6,
 22,